In [ ]:
import sys, xgboost as xgb, sklearn
print("Python:", sys.executable)
print("xgboost:", xgb.__version__)
print("scikit-learn:", sklearn.__version__)

In [ ]:
# Run these in a notebook cell (so it targets the active kernel)
%pip install -U "xgboost>=2.0.0" "scikit-learn>=1.3" "numpy>=1.23" "scipy>=1.7"

In [ ]:
import xgboost as xgb, sklearn, inspect
print("xgboost:", xgb.__version__)
print("scikit-learn:", sklearn.__version__)
print("early_stopping_rounds in XGBRegressor.fit:",
      'early_stopping_rounds' in inspect.signature(xgb.XGBRegressor.fit).parameters)
print("eval_metric in XGBRegressor.__init__:",
      'eval_metric' in inspect.signature(xgb.XGBRegressor.__init__).parameters)

In [ ]:
# %pip install optuna
%pip install -U scikit-learn

In [ ]:
%pip install nbformat

In [3]:
import warnings, json
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error
from catboost import CatBoostRegressor, Pool

import optuna
import xgboost as xgb
from xgboost.callback import EarlyStopping as XgbEarlyStopping

import optuna.visualization as vis


In [4]:
# Config

RANDOM_STATE = 42
N_SPLITS = 5
EARLY_STOP = 200
VERBOSE_TUNE = 0

CV_KIND = 'timeseries'
SHUFFLE = False

np.random.seed(RANDOM_STATE)

In [5]:
TRAIN_PATH = "./data/train_combined.csv"
TEST_PATH = "./data/test_combined.csv"
ID_COL = "id"

TARGETS = ["Y1", "Y2"]
DROP_COLS = [ID_COL]


In [6]:
df_train = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH)

feature_cols = [c for c in df_train.columns if c not in set(TARGETS + DROP_COLS)]
cat_cols = []
num_cols = [c for c in feature_cols]

X = df_train[feature_cols].reset_index(drop=True)
test_X = df_test[feature_cols].reset_index(drop=True)
Y = {t: df_train[t].reset_index(drop=True) for t in TARGETS}
cat_idx = []

In [7]:
def get_cv():
    if CV_KIND == "timeseries":
        return TimeSeriesSplit(n_splits=N_SPLITS)
    return KFold(n_splits=N_SPLITS, shuffle=SHUFFLE, random_state=RANDOM_STATE)

cv = get_cv()

In [8]:
def tune_catboost_for_target(y_col, n_trials=60, use_gpu=False):
    y = Y[y_col]

    def objective(trial):
        params = {
            "loss_function": "RMSE",
            "eval_metric": "R2",
            "depth": trial.suggest_int("depth", 4, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 20.0, log=True),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 5.0),
            "random_strength": trial.suggest_float("random_strength", 0.0, 2.0),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 64),
            "n_estimators": trial.suggest_int("n_estimators", 800, 2500, step=100),
            "allow_writing_files": False,
            "random_state": RANDOM_STATE,
            "verbose": VERBOSE_TUNE
        }
        if use_gpu:
            params["task_type"] = "GPU"

        scores = []
        for fold, (tr_idx, va_idx) in enumerate(cv.split(X, y)):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
            y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

            model = CatBoostRegressor(**params)
            model.fit(
                X_tr, y_tr,
                cat_features=[],
                eval_set=(X_va, y_va),
                early_stopping_rounds=EARLY_STOP,
            )
            pred = model.predict(X_va)
            scores.append(r2_score(y_va, pred))

            trial.report(float(np.mean(scores)), step=fold)
            if trial.should_prune():
                raise optuna.TrialPruned()
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE), pruner=optuna.pruners.MedianPruner(n_warmup_steps=1))

    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    return study


In [9]:
N_TRIALS_CAT = 200
studies = {}
for tgt in TARGETS:
    print(f"=== Tuning CatBoost for {tgt} ===")
    st = tune_catboost_for_target(tgt, n_trials=N_TRIALS_CAT, use_gpu=False)
    print(f"[{tgt}] Best CV R2: {st.best_value:.5f}")
    print(f"[{tgt}] Best Params: {st.best_trial.params}\n")
    studies[tgt] = st

[I 2025-09-22 21:12:20,055] A new study created in memory with name: no-name-8e61c97f-583b-426d-b0d8-2280bfd7ab19


=== Tuning CatBoost for Y1 ===


Best trial: 0. Best value: 0.723339:   0%|          | 1/200 [00:02<09:28,  2.86s/it]

[I 2025-09-22 21:12:22,929] Trial 0 finished with value: 0.7233392515965729 and parameters: {'depth': 6, 'learning_rate': 0.2536999076681771, 'l2_leaf_reg': 8.960785365368121, 'bagging_temperature': 2.993292420985183, 'random_strength': 0.31203728088487304, 'min_data_in_leaf': 10, 'n_estimators': 900}. Best is trial 0 with value: 0.7233392515965729.


Best trial: 1. Best value: 0.727075:   1%|          | 2/200 [00:18<34:14, 10.38s/it]

...

Best trial: 93. Best value: 0.681587:  62%|██████▏   | 124/200 [13:34<09:09,  7.23s/it]

[I 2025-09-22 21:54:27,475] Trial 123 pruned. 


Best trial: 93. Best value: 0.681587:  62%|██████▏   | 124/200 [13:44<08:25,  6.65s/it]


[W 2025-09-22 21:54:37,433] Trial 124 failed with parameters: {'depth': 9, 'learning_rate': 0.02877272080493742, 'l2_leaf_reg': 14.547307603415014, 'bagging_temperature': 2.1370405510512716, 'random_strength': 0.44449468281733195, 'min_data_in_leaf': 3, 'n_estimators': 2000} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/6v/1zrv_7_d0zzd8j18ssjr64ch0000gn/T/ipykernel_9557/344047055.py", line 28, in objective
    model.fit(
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/catboost/core.py", line 5873, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, graph, sample_weight, None, None, None, None, baseline,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^

KeyboardInterrupt: 

In [ ]:
fig = vis.plot_optimization_history(st)
    fig.show()


In [ ]:
# print(studies["Y1"].best_trial.params)
# print(studies["Y2"].best_trial.params)

cat_best_params = {"Y1": {'depth': 8, 'learning_rate': 0.040727522856511406, 'l2_leaf_reg': 1.0333160687802394, 'bagging_temperature': 3.8381299885039564, 'random_strength': 1.177785615607334, 'min_data_in_leaf': 1, 'n_estimators': 1400},
                   "Y2": {'depth': 8, 'learning_rate': 0.04557837932439958, 'l2_leaf_reg': 1.003216622307434, 'bagging_temperature': 2.8629839499568, 'random_strength': 0.30214757243346213, 'min_data_in_leaf': 34, 'n_estimators': 1300}
                   }
"""
{'depth': 8, 'learning_rate': 0.040727522856511406, 'l2_leaf_reg': 1.0333160687802394, 'bagging_temperature': 3.8381299885039564, 'random_strength': 1.177785615607334, 'min_data_in_leaf': 1, 'n_estimators': 1400}
{'depth': 8, 'learning_rate': 0.04557837932439958, 'l2_leaf_reg': 1.003216622307434, 'bagging_temperature': 2.8629839499568, 'random_strength': 0.30214757243346213, 'min_data_in_leaf': 34, 'n_estimators': 1300}
"""

In [13]:
def train_oof_catboost(y_col, params, use_gpu=False):
    y = Y[y_col].values
    oof = np.zeros(len(X))
    test_pred = np.zeros(len(test_X))
    models = []

    p = dict(params)
    p.update({
        "loss_function": "RMSE",
        "eval_metric": "R2",
        "allow_writing_files": False,
        "verbose": 0
    })
    if use_gpu:
        p["task_type"] = "GPU"
    
    for fold, (tr_idx, va_idx) in enumerate(get_cv().split(X, y)):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y[tr_idx], y[va_idx]

        model = CatBoostRegressor(**p, random_state=RANDOM_STATE + fold)
        model.fit(
            X_tr, y_tr,
            cat_features = [],
            eval_set = (X_va, y_va),
            early_stopping_rounds=EARLY_STOP,
        )
        oof[va_idx] = model.predict(X_va)
        test_pred += model.predict(test_X) / N_SPLITS
        models.append(model)
        print(f"[{y_col}] Fold {fold+1}/{N_SPLITS} complete.")

    cv_r2 = r2_score(Y[y_col], oof)
    mse  = mean_squared_error(Y[y_col], oof)
    rmse = float(np.sqrt(mse))
    print(f"[OOF] {y_col}: R2={cv_r2:.5f} | RMSE={rmse:.5f}")
    return models, oof, test_pred, cv_r2

In [14]:
train_oof_catboost("Y1", {'depth': 8, 'learning_rate': 0.037770406084566416, 'l2_leaf_reg': 19.89034552682187, 'bagging_temperature': 2.9239327111077285, 'random_strength': 0.18761743934840397, 'min_data_in_leaf': 50, 'n_estimators': 2100}
, use_gpu=False)


[Y1] Fold 1/5 complete.
[Y1] Fold 2/5 complete.
[Y1] Fold 3/5 complete.
[Y1] Fold 4/5 complete.
[Y1] Fold 5/5 complete.
[OOF] Y1: R2=0.63650 | RMSE=0.58521


([<catboost.core.CatBoostRegressor at 0x13fe12840>,
 array([ 0.        ,  0.        ,  0.        , ..., -0.19933465,
        -0.56869217, -0.55460385], shape=(80000,)),
 array([ 0.40238588, -0.18884709, -0.16702734, ...,  0.24241503,
         0.26853415,  0.27010785], shape=(15996,)),
 0.6365043697560494)

In [ ]:
cat_models = {}
oof_preds = {}
test_preds = {}
cv_scores = {}

for tgt in TARGETS:
    best_params = cat_best_params[tgt]
    models, oof, test_p, r2cv = train_oof_catboost(tgt, best_params, use_gpu=False)
    cat_models[tgt] = models
    oof_preds[tgt] = oof
    test_preds[tgt] = test_p
    cv_scores[tgt] = r2cv

print("CV Scores:", cv_scores)

In [ ]:
submission = pd.DataFrame()
submission[ID_COL] = df_test[ID_COL]
for tgt in TARGETS:
    submission[tgt] = test_preds[tgt]

OUT_PATH = "submission_catboost_optuna.csv"
submission.to_csv(OUT_PATH, index=False)
print("Saved:", OUT_PATH)
submission.head()

now test xgb boost and blend both together  

In [10]:
def _get_best_iteration(bst):
    # Works across xgboost 1.x/2.x/3.x
    bi = getattr(bst, "best_iteration", None)
    if bi is None:
        # some builds store it in attributes dict
        try:
            bi = int(bst.attributes().get("best_iteration"))
        except Exception:
            bi = None
    return bi

def _predict_best(bst, dmatrix):
    bi = _get_best_iteration(bst)
    if bi is not None:
        # in 3.x use iteration_range instead of ntree_limit
        return bst.predict(dmatrix, iteration_range=(0, bi + 1))
    # fallback: use all trees
    return bst.predict(dmatrix)
    
def tune_xgb_for_target(y_col, n_trials=60, use_gpu=False):
    y = Y[y_col]

    def objective(trial):
        num_boost_round = trial.suggest_int("num_boost_round", 800, 2500, step=100)
        params = {
            "objective": "reg:squarederror",
            "eval_metric": "rmse",
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "min_child_weight": trial.suggest_float("min_child_weight", 1.0, 20.0, log=True),
            "alpha": trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True),
            "lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
            "gamma": trial.suggest_float("gamma", 0.0, 5.0),
            "tree_method": "gpu_hist" if use_gpu else "hist",
            "seed": RANDOM_STATE,
        }
        
        scores = []
        for fold, (tr_idx, va_idx) in enumerate(get_cv().split(X, y)):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
            y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

            dtr = xgb.DMatrix(X_tr, label=y_tr)
            dva = xgb.DMatrix(X_va, label=y_va)

            bst = xgb.train(
                params=params,
                dtrain=dtr,
                num_boost_round=num_boost_round,
                evals=[(dva, "valid")],
                early_stopping_rounds=EARLY_STOP,
                verbose_eval=False,
            )

            pred = _predict_best(bst, dva)
            scores.append(r2_score(y_va, pred))

            trial.report(float(np.mean(scores)), step=fold)
            if trial.should_prune():
                raise optuna.TrialPruned()
            
        return float(np.mean(scores))
    
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE), pruner=optuna.pruners.MedianPruner(n_warmup_steps=1),)
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)
    return study

In [11]:
N_TRIALS_XGB = 200

xgb_studies = {}
for tgt in TARGETS:
    print(f"=== Tuning XGBoost for {tgt} ===")
    st = tune_xgb_for_target(tgt, n_trials=N_TRIALS_XGB, use_gpu=False)
    print(f"[{tgt}] Best CV R2 (XGB): {st.best_value:.5f}")
    print(f"[{tgt}] Best Params (XGB): {st.best_trial.params}\n")
    xgb_studies[tgt] = st

[I 2025-09-22 21:55:17,035] A new study created in memory with name: no-name-1ab0a6ee-3436-478f-b0aa-263c63c0884f


=== Tuning XGBoost for Y1 ===


Best trial: 0. Best value: 0.710878:   0%|          | 1/200 [00:01<04:09,  1.25s/it]

[I 2025-09-22 21:55:18,287] Trial 0 finished with value: 0.7108779217530772 and parameters: {'num_boost_round': 1400, 'learning_rate': 0.2536999076681771, 'max_depth': 8, 'subsample': 0.7993292420985183, 'colsample_bytree': 0.5780093202212182, 'min_child_weight': 1.5957084694148351, 'reg_alpha': 2.9152036385288193e-08, 'reg_lambda': 0.6245760287469887, 'gamma': 3.005575058716044}. Best is trial 0 with value: 0.7108779217530772.


Best trial: 1. Best value: 0.725204:   1%|          | 2/200 [00:11<20:52,  6.33s/it]

...

Best trial: 151. Best value: 0.681543:  76%|███████▋  | 153/200 [06:29<01:47,  2.28s/it]

[I 2025-09-22 22:22:39,194] Trial 152 pruned. 


Best trial: 151. Best value: 0.681543:  76%|███████▋  | 153/200 [06:30<01:59,  2.55s/it]

[W 2025-09-22 22:22:39,864] Trial 153 failed with parameters: {'num_boost_round': 900, 'learning_rate': 0.05953949203423973, 'max_depth': 10, 'subsample': 0.7080653832235071, 'colsample_bytree': 0.6568239255401569, 'min_child_weight': 15.621867287856906, 'reg_alpha': 0.7358996659880034, 'reg_lambda': 7.063769097838879, 'gamma': 2.6185167968393763} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/6v/1zrv_7_d0zzd8j18ssjr64ch0000gn/T/ipykernel_9557/1144022015.py", line 48, in objective
    bst = xgb.train(
          ^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/core.py", line 729, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "/Library/Framew

KeyboardInterrupt: 

In [ ]:
print(xgb_studies["Y1"].best_trial.params)
print(xgb_studies["Y2"].best_trial.params)

"""
{'num_boost_round': 900, 'learning_rate': 0.011387425172293873, 'max_depth': 9, 'subsample': 0.5642895080936234, 'colsample_bytree': 0.5581569764539639, 'min_child_weight': 19.703853846581026, 'reg_alpha': 3.336634822084743e-07, 'reg_lambda': 1.129627556934382e-07, 'gamma': 0.4896075810838035}
{'num_boost_round': 2300, 'learning_rate': 0.01354184424412271, 'max_depth': 9, 'subsample': 0.7339248014853161, 'colsample_bytree': 0.7407604054011988, 'min_child_weight': 1.392834443288169, 'reg_alpha': 1.693305529699566e-05, 'reg_lambda': 3.897250984223176e-06, 'gamma': 0.16970161657112237}
"""

In [15]:
def train_oof_xgb(y_col, best_params, use_gpu=False):
    y = Y[y_col].values
    oof = np.zeros(len(X))
    test_pred = np.zeros(len(test_X))
    model = []

    num_boost_round = int(best_params.get("n_estimators", 1200))
    params = {
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "eta": float(best_params["learning_rate"]),
        "max_depth": int(best_params["max_depth"]),
        "subsample": float(best_params["subsample"]),
        "colsample_bytree": float(best_params["colsample_bytree"]),
        "min_child_weight": float(best_params["min_child_weight"]),
        "alpha": float(best_params["reg_alpha"]),
        "lambda": float(best_params["reg_lambda"]),
        "gamma": float(best_params["gamma"]),
        "tree_method": "gpu_hist" if use_gpu else "hist",
        "seed": RANDOM_STATE,
    }

    dtest = xgb.DMatrix(test_X)

    for fold, (tr_idx, va_idx) in enumerate(get_cv().split(X, y)):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y[tr_idx], y[va_idx]

        dtr = xgb.DMatrix(X_tr, label=y_tr)
        dva = xgb.DMatrix(X_va, label=y_va)

        bst = xgb.train(
            params=params,
            dtrain=dtr,
            num_boost_round=num_boost_round,
            evals=[(dva, "valid")],
            early_stopping_rounds=EARLY_STOP,
            verbose_eval=False,
        )

        oof[va_idx] = _predict_best(bst, dva)
        test_pred += _predict_best(bst, dtest) / N_SPLITS
        model.append(bst)
        print(f"[{y_col}] Fold {fold+1}/{N_SPLITS} complete.")

    cv_r2 = r2_score(Y[y_col], oof)
    rmse = float(np.sqrt(mean_squared_error(Y[y_col], oof)))
    print(f"[OOF] {y_col}: R2={cv_r2:.5f} | RMSE={rmse:.5f}")
    return model, oof, test_pred, cv_r2

In [16]:
{'num_boost_round': 1600, 'learning_rate': 0.016851099464040075, 'max_depth': 10, 'subsample': 0.5081713485200369, 'colsample_bytree': 0.7440582265712843, 'min_child_weight': 9.618965261856685, 'reg_alpha': 1.2340026835726136e-06, 'reg_lambda': 1.2201979893309862, 'gamma': 1.0103341566498545}
train_oof_xgb("Y1", {'num_boost_round': 1600, 'learning_rate': 0.016851099464040075, 'max_depth': 10, 'subsample': 0.5081713485200369, 'colsample_bytree': 0.7440582265712843, 'min_child_weight': 9.618965261856685, 'reg_alpha': 1.2340026835726136e-06, 'reg_lambda': 1.2201979893309862, 'gamma': 1.0103341566498545}, use_gpu=False)

[Y1] Fold 1/5 complete.
[Y1] Fold 2/5 complete.
[Y1] Fold 3/5 complete.
[Y1] Fold 4/5 complete.
[Y1] Fold 5/5 complete.
[OOF] Y1: R2=0.64402 | RMSE=0.57913


([<xgboost.core.Booster at 0x10f0c7e60>,
 array([ 0.        ,  0.        ,  0.        , ..., -0.1841052 ,
        -0.60279471, -0.53801417], shape=(80000,)),
 array([ 0.4006054 , -0.17397015, -0.17981971, ...,  0.25298231,
         0.24925774,  0.25128531], shape=(15996,)),
 0.6440198658996401)

In [ ]:
xgb_models = {}
oof_xgb = {}
test_xgb = {}
xgb_cv_scores = {}

for tgt in TARGETS:
    best_params_xgb = xgb_studies[tgt].best_trial.params
    models, oof, te, r2cv = train_oof_xgb(tgt, best_params_xgb, use_gpu=False)
    xgb_models[tgt] = models
    oof_xgb[tgt] = oof
    test_xgb[tgt] = te
    xgb_cv_scores[tgt] = r2cv

print("XGB CV R2:", xgb_cv_scores)

In [ ]:
oof_cat = oof_preds
test_cat = test_preds

def rmse_score(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

# simple 50 50 blend
oof_blend_avg = {}
test_blend_avg = {}
for tgt in TARGETS:
    oof_blend_avg[tgt] = 0.5 * oof_cat[tgt] + 0.5 * oof_xgb[tgt]
    test_blend_avg[tgt] = 0.5 * test_cat[tgt] + 0.5 * test_xgb[tgt]
    r2 = r2_score(Y[tgt], oof_blend_avg[tgt])
    rmse = rmse_score(Y[tgt], oof_blend_avg[tgt])
    print(f"[BLEND AVG] {tgt}: R2={r2:.5f} | RMSE={rmse:.5f}")

# ridge-learned wieghts 
weights = {}
oof_blend_ridge = {}
test_blend_ridge = {}

for tgt in TARGETS:
    Xb_oof = np.vstack([oof_cat[tgt], oof_xgb[tgt]]).T
    ridge = Ridge(alpha=1e-3, fit_intercept=True, random_state=RANDOM_STATE)
    ridge.fit(Xb_oof, Y[tgt])
    w_cat, w_xgb = ridge.coef_
    b = ridge.intercept_
    weights[tgt] = {"w_cat": float(w_cat), "w_xgb": float(w_xgb), "intercept": float(b)}

    oof_blend_ridge[tgt] = Xb_oof @ np.array([w_cat, w_xgb]) + b
    print(f"[Ridge Blend] {tgt}: OOF R2={r2_score(Y[tgt], oof_blend_ridge[tgt]):.5f} | RMSE={rmse_score(Y[tgt], oof_blend_ridge[tgt]):.5f} | Weights={weights[tgt]}")

    # Apply to test
    Xb_test = np.vstack([test_cat[tgt], test_xgb[tgt]]).T
    test_blend_ridge[tgt] = Xb_test @ np.array([w_cat, w_xgb]) + b


In [ ]:
rows = []
for tgt in TARGETS:
    rows.append({
        "target": tgt,
        "CatBoost OOF R2": r2_score(Y[tgt], oof_cat[tgt]),
        "XGBoost OOF R2": r2_score(Y[tgt], oof_xgb[tgt]),
        "Avg(0.5/0.5) OOF R2": r2_score(Y[tgt], oof_blend_avg[tgt]),
        "Ridge Blend OOF R2": r2_score(Y[tgt], oof_blend_ridge[tgt]),
    })
pd.DataFrame(rows).sort_values("Ridge Blend OOF R2", ascending=True)

In [ ]:
submission = pd.DataFrame()

submission[ID_COL] = df_test[ID_COL]
for tgt in TARGETS:
    submission[tgt] = test_blend_ridge[tgt]

OUT_PATH = "submission_catboost_xgboost_blend.csv"
submission.to_csv(OUT_PATH, index=False)
print("Saved:", OUT_PATH)
submission.head()